In [1]:
import albumentations.pytorch
import customdataset as ds
import torch as t
import torchvision as tv
import torchvision.tv_tensors as tvt
import albumentations as A

x, y = ds.extract_all('./datasets/090/annotations.xml')
x1, y1 = ds.extract_all('./datasets/190/annotations.xml')
x2, y2 = ds.extract_all('./datasets/30/annotations.xml')
x3, y3 = ds.extract_all('./datasets/60/annotations.xml')
x4, y4 = ds.extract_all('./datasets/90/annotations.xml')

x5, y5 = ds.extract_all('./datasets/clear1/annotations.xml')

train_ds = ds.CustomImageDataset([*x, *x1, *x2, *x3, *x4], [*y, *y1, *y2, *y3, *y4], transform=A.Compose([
    A.Resize(1024, 1024),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    A.ShiftScaleRotate(
        shift_limit=0.0625,  # fraction of image height/width
        scale_limit=0.1,  # +/- 10% scale
        rotate_limit=15,  # degrees
        border_mode=0,  # fill outside with 0 (black)
        p=0.5
    ),
    albumentations.pytorch.ToTensorV2()
],
    bbox_params=ds.albumentations_params)
                                 )

val_ds = ds.CustomImageDataset(x5, y5)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
frame_000088.PNG is empty
frame_000089.PNG is empty
frame_000090.PNG is empty
frame_000091.PNG is empty
frame_000092.PNG is empty
frame_000093.PNG is empty
frame_000094.PNG is empty
frame_000095.PNG is empty
frame_000096.PNG is empty
frame_000097.PNG is empty
frame_000098.PNG is empty
frame_000099.PNG is empty
frame_000100.PNG is empty
frame_000101.PNG is empty
frame_000102.PNG is empty
frame_000103.PNG is empty
frame_000104.PNG is empty
frame_000105.PNG is empty
frame_000106.PNG is empty
frame_000107.PNG is empty
frame_000108.PNG is empty
frame_000109.PNG is empty
frame_000110.PNG is empty
frame_000111.PNG is empty
frame_000112.PNG is empty
frame_000113.PNG is empty
frame_000114.PNG is empty
frame_000115.PNG i

C:\Users\n3\Desktop\od-eval-2\.venv\lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [2]:

device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')

dl_train = t.utils.data.DataLoader(train_ds, batch_size=4, shuffle=True, collate_fn=lambda e: tuple(zip(*e)))
dl_val = t.utils.data.DataLoader(val_ds, batch_size=4, shuffle=True, collate_fn=lambda e: tuple(zip(*e)))

model = tv.models.detection.fasterrcnn_resnet50_fpn(weights='COCO_V1', backbone_weights='IMAGENET1K_V2').to(device)

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = tv.models.detection.faster_rcnn.FastRCNNPredictor(
    in_channels=model.roi_heads.box_predictor.cls_score.in_features,
    num_classes=2
).to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = t.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = t.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)



In [3]:
from tqdm.notebook import tqdm, trange
import torchmetrics as tm

best_val_loss = float('inf')
best_val_map = 0

patience = 5  # stop if val_loss doesn't improve for 5 epochs
counter = 0
best_model_state = None
num_epochs = 100
metric = tm.detection.mean_ap.MeanAveragePrecision("xyxy")

for epoch in range(num_epochs):
    # ---- TRAIN ----
    model.train()
    train_loss_epoch = 0

    dl_train_tqdm = tqdm(dl_train, desc=f"Train Epoch {epoch + 1}", leave=False)

    for images, targets in dl_train_tqdm:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        total_loss = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        train_loss_epoch += total_loss.item()

        dl_train_tqdm.set_postfix(loss=total_loss.item())

    train_loss_epoch /= len(dl_train)

    # ---- VALIDATION ----
    model.eval()
    val_loss_epoch = 0
    metric.reset()

    with t.no_grad():

        dl_val_tqdm = tqdm(dl_val, desc=f"Val Epoch {epoch + 1}", leave=False)

        for images, targets in dl_val_tqdm:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            pred = model(images)

            metric.update(pred, targets)

    val_metrics = metric.compute()
    val_map = val_metrics["map"].item()
    print(f"Epoch {epoch + 1} | Train Loss: {train_loss_epoch:.4f} | Val mAP: {val_map:.4f}")

    # ---- Early Stopping ----
    if val_map > best_val_map:
        best_val_map = val_map
        best_model_state = model.state_dict()
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            model.load_state_dict(best_model_state)
            break


Train Epoch 1:   0%|                                                         | 0/81 [00:00<?, ?it/s]

C:\Users\n3\Desktop\od-eval-2\customdataset.py:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  areas = t.tensor(areas, dtype=t.float32)


Val Epoch 1:   0%|                                                            | 0/1 [00:00<?, ?it/s]

Epoch 1 | Train Loss: 0.2295 | Val mAP: 0.3997


Train Epoch 2:   0%|                                                         | 0/81 [00:00<?, ?it/s]

...

Train Epoch 5:   0%|                                                         | 0/81 [00:00<?, ?it/s]

Val Epoch 5:   0%|                                                            | 0/1 [00:00<?, ?it/s]

Epoch 5 | Train Loss: 0.0652 | Val mAP: 0.3946


Train Epoch 6:   0%|                                                         | 0/81 [00:00<?, ?it/s]

AssertionError: Expected target boxes to be a tensor of shape [N, 4], got torch.Size([0]).